In [2]:
# This is its own cell because it takes a while to load this thing
from gensim import models

# takes a little bit. increase limit at own risk.
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [113]:
# imports and list of scrape times.

import requests
import json
import pickle
import os
import numpy as np
import time
import shutil
import datetime
import cv2
import urllib.request
import sqlite3 as sql

list_of_scrape_times = []

In [196]:
## class Loader

class Loader:
    def __init__(self, list_of_ids = [], list_of_times = []):
        self.list_of_ids = list_of_ids
        self.list_of_times = list_of_times
    
    filedir = ""

    ## 'file' omitted because it's not used in generating features
    list_of_types = ['text','log','contact','calender','gps','tweets','Instagram', 'Instagram media', 'audio','phq']
    
    def deletdatboi(self, nombre):
        shutil.rmtree('./datafor' + list_of_scrape_times[nombre])
        
    def lids(self):
        return self.list_of_ids
    
    def lits(self):
        return self.list_of_times
    
    def downloadAndLabel(self):
        # connect to file
        conn = sql.connect('phonedata.db')
        # create cursor for making calls to database
        c = conn.cursor()

        for row in c.execute('SELECT * FROM ids'):
            # one result
            self.list_of_ids.append(row[0])
            self.list_of_times.append(row[1])

        number_cols = len(self.list_of_types)
        number_rows = len(self.list_of_ids)

        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        timereadable = datetime.datetime.fromtimestamp(int(timenow)).strftime('%H:%M')
        list_of_scrape_times.append(timereadable)
        os.mkdir('./datafor' + timereadable)
        Loader.filedir = './datafor' + timereadable

        for i in range(0,number_rows):
            for j in range(0,number_cols):

                # print(self.list_of_ids[i] + " "  + self.list_of_types[j])
                
                apickle = []

                exampleLookup = (self.list_of_ids[i], self.list_of_types[j])
                for row in c.execute('SELECT DISTINCT* FROM data WHERE id=? AND type=?', exampleLookup):
                    # the row with the ID, type, and content
                    apickle.append(row[2])

                # print(apickle)    
                
                pickle.dump(apickle, open( Loader.filedir  + "/DP" + self.list_of_ids[i] +  self.list_of_types[j] + ".p", "wb" ))
        
        ## OLD CODE FOR INTERFACING THROUGH THE WEBSERVER
        ## CODE ABOVE ACCESSES A .DB FILE LOCALLY
        '''
        
        r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
        list_of_idtime = r.json()
        

        for i in range(0,len(list_of_idtime)):
            self.list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )

        for i in range(0,len(list_of_idtime)):
            self.list_of_times.append( list_of_idtime[i]['date'] )

        number_cols = len(self.list_of_types)
        number_rows = len(self.list_of_ids)
        
        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        timereadable = datetime.datetime.fromtimestamp(int(timenow)).strftime('%H:%M')
        list_of_scrape_times.append(timereadable)
        os.mkdir('./datafor' + timereadable)
        Loader.filedir = './datafor' + timereadable


        for i in range(0,number_rows):
            for j in range(0,number_cols):
                
                temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(self.list_of_ids[i])) + '&type=' + self.list_of_types[j])
                fintemp = json.loads(temp.text)["data"]
                
                while(json.loads(temp.text)["nextURL"] != ''):
                    temp = requests.get('http://depressionmqp.wpi.edu:8080' + json.loads(temp.text)["nextURL"])
                    fintemp += json.loads(temp.text)["data"]
                
                pickle.dump(fintemp, open( Loader.filedir + "/DP" + str(int(self.list_of_ids[i])) +  self.list_of_types[j] + ".p", "wb" ))

                # loads it into memory. i will not use this for now
                # for sake of architectural sanity
                # list_of_jsons[i][j] = temp
        '''

In [166]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in text pickle and scrapedate, spits out text frequency
    def dailyTextFreq(self, texts, scrapedate):
        
        a = texts

        seconds_intwo_weeks = 1209600;
        
        # calculate unix time for two weeks prior
        two_weeks_prior = scrapedate - seconds_intwo_weeks

        text_date = 0
        saved_index = 0

        # find the index of the oldest message that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < text_date):
                saved_index = len(a) - (i+1)
                break

        # divide by 14 to get daily freq
        return float(saved_index+1)/14
    

    # takes in call pickle and scrapedate, spits out call frequency
    def dailyCallFreq(self, calls, scrapedate):

        a = calls

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        call_date = 0
        saved_index = 0

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < call_date):
                saved_index = len(a) - (i+1)
                break

        return float(saved_index+1)/14
    
    
    # input is tweets pickle, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(tweets)):
            try:
                masterSum += self.tweetToEmbedding(tweets[i])
            except:
                pass
        
        return masterSum

    
    # input is one single tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = tweet['text'].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # input is tweets pickle, returns follow count
    def followerCount(self, tweets):

        followerCount = 0

        # get
        try:
            followerCount = json.loads(tweets[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    # input is tweets pickle, returns friend count
    def followingCount(self, tweets):

        followingCount = 0

        try:
            followingCount = json.loads(tweets[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    # input is tweets pickle, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        a = tweets

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_likes = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets[i])['favorite_count']


        return sum_of_likes/14

    # input is tweets pickle, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        a = tweets

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_retweets = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets[i])['retweet_count']


        return sum_of_retweets/14
    
    # input is contacts pickle, returns number of contacts
    def numOfContacts(self, contacts):
        return len(contacts)
    
    # input is instagram pickle, return two features: follows count, followed by count  
    def instagramThings(self, instagram):
        
        followsFollowed = np.zeros((2,))
        
        
        try:
            if(type(json.loads(instagram[0])) == str):
                print("DAMN YOU DAMON!!(ig)")
            else: # its a dictionary like it's supposed to be
                followsFollowed[0] = json.loads(instagram[0])['data']['counts']['follows']
                followsFollowed[1] = json.loads(instagram[0])['data']['counts']['followed_by']
        except IndexError:
            pass
        
        return followsFollowed
    
    # takes in instagramMedia pickle scrape date, spits out filter usage frequency for the past 2 weeks
    def instagramFilterFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        negative_count = 0

        for i in range(0,saved_index):
            if(json.loads(a[i])['filter'] == Normal):
                negative_count += 1

        # filter usage freq = posts with filters count / total post count 
        return ((saved_index+1)-negative_count)/(saved_index+1)
    
    # takes in instagramMedia pickle and scrape date, returns a vector that
    # contains a normalized percentage (0-1) for the usage of the filters
    # listed below for the past 2 weeks: 
    # Valencia, X-Pro II, Hefe, Amaro, Rise, Willow, Crema, Inkwell
    
    # output example: [0.5,0,0,0,0,0.5,0,0] 
    # interpretation: user used valencia half the time, Willow the other half
    # of time, for the past 2 weeks of Instagram posts.
    def instagramFilterVector(self, instagramMedia, scrapedate):
        
        filters = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
        filtervec = np.zeros((8,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        for i in range(0,saved_index):
            filt = json.loads(a[i])['filter']
            if(filt in filters):
                filters[filt] += 1
            else:
                pass
            
        for i in range(0,8):
            filtervec[i] = filters[list(filters)[i]]

        # percentage of how much the filters are used as a normalized vector
        return filtervec/(saved_index+1)
    
    # takes in InstagramMedia, returns comment and like frequency for the 
    # past 2 weeks.
    def instagramLikeComFreq(self, instagramMedia, scrapedate):
        
        counts = np.zeros((2,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        for i in range(0,saved_index):
            counts[0] += json.loads(instagramMedia[i])['likes']['count']
            counts[1] += json.loads(instagramMedia[i])['comments']['count']
        
            
        # because dividing by 0 is a big No-No
        if (not(counts[0] != 0 and counts[1] != 0)):
            return np.zeros((2,))
        else:
            # likes per post for past 2 weeks
            return counts/saved_index


    # takes in instagramMedia pickle and scrapedate, returns IG post 
    # frequency for the past 2 weeks
    def instagramPostFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        return (saved_index+1)/14
        
        
    # takes instagramMedia, returns pixelwise average values for [H,S,V]
    # (Hue, Saturation, Value) for all posts in the past 2 weeks, the 
    # count of faces as a frequency of faces per picture, for the past
    # 2 weeks as well.
    def averageHSVF(self, instagramMedia, scrapedate):


        seconds_intwo_weeks = 1209600;

        ## ms into secs
        scrapedate = int(str(scrapedate)[:-3]) 

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0

        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass

            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break


        

        if(os.path.exists("./ILLSTOPBLINKINGSOON")):
            shutil.rmtree('./ILLSTOPBLINKINGSOON')

        os.mkdir('./ILLSTOPBLINKINGSOON')

        avgs = np.zeros((4,))

        ## to avoid division by 0
        if(saved_index == 0):
            return avgs



        for i in range(0,saved_index):
            url = json.loads(instagramMedia[i])['images']['thumbnail']['url']
            urllib.request.urlretrieve(url, './ILLSTOPBLINKINGSOON/' + str(i) + '.jpg')


        # face_cascade here is a pre trained classifier for frontal faces 
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

            
        avgHue = 0
        avgSatur = 0
        avgVal = 0
        avgFaces = 0

        ## GODS OF PROGRAMMING, FORGIVE ME FOR THIS TRIPLE NEST

        for k in range(0, saved_index):
            ## BGR and not RGB because imread reads in BGR
            img = cv2.imread('./ILLSTOPBLINKINGSOON/' + str(k) + '.jpg')
            hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            
            grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
            
            avgFaces += len(faces)
            
            for i in range(0,hsv.shape[0]):
                for j in range(0,hsv.shape[1]):
                    avgHue += hsv[i,j,0]
                    avgSatur += hsv[i,j,1]
                    avgVal += hsv[i,j,2]
                    
        

        sums = [avgHue,avgSatur,avgVal]

        ## 22500 = 150x150 = instagram photo thumbnail shape
        avgs = list(map(lambda x: x/(22500*saved_index), sums)).append(avgFaces/saved_index)

        shutil.rmtree('./ILLSTOPBLINKINGSOON')

        return avgs      

    
    #input is texts pickle, return master vector 
    def embeddingToMastersumText(self, texts):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(texts)):
            try:
                masterSum += self.textToEmbedding(texts[i])
            except:
                pass
        
        return masterSum

    
    # input is single text, returns vector embedding of entire text.
    def textToEmbedding(self, text):
        
        q = json.loads(text)["body"].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # returns [q1,q2,q3,q4,q5,q6,q7,q8,q9] and sum of all these scores
    def labelGenerator(self, phq):
        
        labelVector = np.zeros((10,))
        sumOfScores = 0
        
        print(phq)
        
        for i in range(0,9):
            temp = int(json.loads(phq[0])['Q' + str(i)])
            labelVector[i] = temp
            labelVector[9] += temp
            

In [171]:
class Generator:

    def __init__(self, filedir, list_of_ids, list_of_times):
        self.filedir = filedir
        self.list_of_ids = list_of_ids
        self.f = Featurizer()
        self.num_of_people = len(list_of_ids)
        self.list_of_times = list_of_times
    
    featureMatrix = None
    
    def generateMatrix(self):

        # these are the feature vectors, one for every feature.
        # each feature vector contains the value for one feature
        # for every user. 
        featureVectorCF = np.zeros((self.num_of_people,1))
        featureVectorTF = np.zeros((self.num_of_people,1))
        featureVectorTW = np.zeros((self.num_of_people,300))
        featureVectorFC = np.zeros((self.num_of_people,1))
        featureVectorFC2 = np.zeros((self.num_of_people,1))
        featureVectorTWL = np.zeros((self.num_of_people,1))
        featureVectorTWRT = np.zeros((self.num_of_people,1))
        featureVectorNC = np.zeros((self.num_of_people,1))
        featureVectorIG1 = np.zeros((self.num_of_people,2))
        featureVectorIG2 = np.zeros((self.num_of_people,1))
        featureVectorIGFV = np.zeros((self.num_of_people,8))
        featureVectorIGLC = np.zeros((self.num_of_people,2))
        featureVectorIG3 = np.zeros((self.num_of_people,1))
        featureVectorHSVF = np.zeros((self.num_of_people,4))
        featureVectorTW2V = np.zeros((self.num_of_people,300))
        
        labelVector = np.zeros((self.num_of_people,10))
        
        for i in range(0, self.num_of_people):

            #list_of_jsons[i][1]

                
            a1 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "log" + ".p", "rb" ))    
            a2 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "text" + ".p", "rb" )) 
            a3 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "tweets" + ".p", "rb" )) 
            a4 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "contact" + ".p", "rb" )) 
            a5 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "Instagram" + ".p", "rb" )) 
            a6 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "Instagram media" + ".p", "rb" ))
            
            b1 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "phq" + ".p", "rb" )) 
            
            
            if((len(a1) + len(a2) + len(a3) + len(a4) + len(a5) + len(a6) + len(b1)) == 0):
                # false entry in ID list
                continue
            
            # print(self.list_of_ids[i])
            # print(i)
            
            featureVectorCF[i] = self.f.dailyCallFreq(a1, self.list_of_times[i])
            featureVectorTF[i] = self.f.dailyTextFreq(a2, self.list_of_times[i])
            featureVectorTW[i] = self.f.embeddingToMastersum(a3)
            featureVectorFC[i] = self.f.followerCount(a3)
            featureVectorFC2[i] = self.f.followingCount(a3)
            featureVectorTWL[i] = self.f.twitterLikeFreq(a3, self.list_of_times[i])
            featureVectorTWRT[i] = self.f.twitterRetweetFreq(a3, self.list_of_times[i])
            featureVectorNC[i] = self.f.numOfContacts(a4)
            featureVectorIG1[i] = self.f.instagramThings(a5)
            featureVectorIG2[i] = self.f.instagramFilterFreq(a6, self.list_of_times[i])
            featureVectorIGFV[i] = self.f.instagramFilterVector(a6, self.list_of_times[i])
            featureVectorIGLC[i] = self.f.instagramLikeComFreq(a6, self.list_of_times[i])
            featureVectorIG3[i] = self.f.instagramPostFreq(a6, self.list_of_times[i])
            featureVectorHSVF[i] = self.f.averageHSVF(a6, self.list_of_times[i])
            featureVectorTW2V[i] = self.f.embeddingToMastersumText(a2)
            
            labelVector[i] = self.f.labelGenerator(b1)

            Generator.featureMatrix = np.hstack((featureVectorCF,featureVectorTF,
featureVectorTW, featureVectorFC, featureVectorTWL, featureVectorTWRT,
featureVectorNC,featureVectorIG1, featureVectorIG2, featureVectorIGFV,
featureVectorIGLC, featureVectorIG3, featureVectorHSVF, featureVectorTW2V,
labelVector))


In [197]:
# Everything I coded in one cell!

l = Loader()
l.downloadAndLabel()

# g = Generator(l.filedir, l.lids(), l.lits())
# g.generateMatrix()
# g.featureMatrix.shape

In [183]:
for i in l.lids()[17:18]:
    b1 = pickle.load( open( "datafor15:06" + "/DP" + i +  "phq" + ".p", "rb" )) 
    print(b1)

['{"Q0":"2","Q1":"0","Q2":"1","Q3":"0","Q4":"0","Q5":"1","Q6":"0","Q7":"1","Q8":"1"}', '{"Q0":"2","Q1":"0","Q2":"1","Q3":"0","Q4":"0","Q5":"1","Q6":"0","Q7":"1","Q8":"1"}']


In [189]:
l.lids()[17]

'7835'

In [190]:
b1 = pickle.load( open( "datafor15:06" + "/DP" + "7835" +  "phq" + ".p", "rb" )) 

print(b1)

['{"Q0":"2","Q1":"0","Q2":"1","Q3":"0","Q4":"0","Q5":"1","Q6":"0","Q7":"1","Q8":"1"}', '{"Q0":"2","Q1":"0","Q2":"1","Q3":"0","Q4":"0","Q5":"1","Q6":"0","Q7":"1","Q8":"1"}']


In [194]:
apickle = []
exampleLookup = ("7835", "phq")
for row in c.execute('SELECT DISTINCT * FROM data WHERE id=? AND type=? ', exampleLookup):
    # the row with the ID, type, and content
    apickle.append(row[2])

In [195]:
apickle


['{"Q0":"2","Q1":"0","Q2":"1","Q3":"0","Q4":"0","Q5":"1","Q6":"0","Q7":"1","Q8":"1"}']

In [24]:
## WASTELAND OF PLAY CELLS BELOW
import pickle

filedir = './datafor' + '288'

## access example

# instagram = pickle.load( open( filedir + "/DP" + str(int(l.lids()[4])) +  "tweets" + ".p", "rb" )) 

a3 = pickle.load( open( filedir + "/DP" + '1995377' +  "Instagram media" + ".p", "rb" )) 
# a3 = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
#instagram = pickle.load( open( filedir + "/DP" + '11852603' +  "Instagram" + ".p", "rb" )) 



# json.loads(a3.json()[0])

a3.json()





[]

In [112]:
import requests
import json



# temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(98945548) + '&type=' + "text")
# fintemp = json.loads(temp.text)["data"]

# while(json.loads(temp.text)["nextURL"] != ''):
#     temp = requests.get('http://depressionmqp.wpi.edu:8080' + json.loads(temp.text)["nextURL"])
#     fintemp += json.loads(temp.text)["data"]

    
fintemp[8817]
    

'{"_id":"4","thread_id":"1","address":" 15085301734","person":"null","date":"1460240309687","date_sent":"0","protocol":"null","read":"1","status":"-1","type":"2","reply_path_present":"null","subject":"null","body":"Your face","service_center":"null","locked":"0","error_code":"0","seen":"1","deletable":"0","sim_slot":"0","sim_imsi":"null","hidden":"0","group_id":"null","group_type":"null","delivery_date":"null","app_id":"0","msg_id":"0","callback_number":"null","reserved":"0","pri":"0","teleservice_id":"0","link_url":"null","svc_cmd":"0","svc_cmd_content":"null","roam_pending":"0","spam_report":"0","safe_message":"0","sub_id":"-1","creator":"com.android.mms","secret_mode":"0","favorite":"0","d_rpt_cnt":"0","using_mode":"0","from_address":"null","announcements_subtype":"0","announcements_scenario_id":"null","device_name":"null"}'

In [525]:
kelsey = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}

# for fil in kelsey:
#     print(kelsey[fil])

'Valencia' in kelsey

True

In [552]:
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
filters = {'Lark':0,'Slumber':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
filtervec = np.ones((8,))

for i in range(0,3):
    filt = json.loads(a[i])['filter']
    if(filt in filters):
        filters[filt] += 1
    else:
        pass
    
for i in range(0,8):
    filtervec[i] = filters[list(filters)[i]]
    
filtervec

array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.])

In [613]:
## TWITTER 70522438
## IG   19671950


# instagramMedia = pickle.load( open( filedir + "/DP" + '82562975' +  "Instagram media" + ".p", "rb" )) 
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 

url = json.loads(instagramMedia.json()[0])['images']['thumbnail']['url']

from PIL import Image
import requests
from io import BytesIO

response = requests.get(url)
img = Image.open(BytesIO(response.content))


img.shape

AttributeError: 'JpegImageFile' object has no attribute 'shape'

In [520]:
filtervec = np.ones((8,))
filtervec/4

array([ 0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25])

In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [156]:
timenow  = str(int(time.time())) # for temporal congruency
timenow1 = timenow
timenow1

'1511765588'

In [125]:
l.lits()

[1513024716885,
 1513026200334,
 1513026939825,
 1513027798716,
 1513028059396,
 1513028060408,
 1513029109178,
 1513034337824,
 1513034463419,
 1513035395273]

In [184]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


image = cv2.imread('./faces.jpg')
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#faces = face_cascade.detectMultiScale(grayImage)
faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
print(len(faces))

#type(face_cascade) 

10


In [178]:
dir_file_list = os.listdir(".")
dir_file_list

['datafor19:07',
 'MLIntP3.ipynb',
 'ILLSTOPBLINKINGSOON',
 '.git',
 '.ipynb_checkpoints',
 'OpenCV Shenaniganry.ipynb',
 'README.md',
 'haarcascade_frontalface_default.xml',
 'faces.jpg',
 'glove.twitter.27B.zip',
 'MLInt.ipynb',
 'datafor690',
 'glove.twitter.27B.50d.txt',
 'GoogleNews-vectors-negative300.bin']

In [67]:
import sqlite3 as sql

# connect to file
conn = sql.connect('phonedata.db')
# create cursor for making calls to database
c = conn.cursor()

list_of_ids = []
list_of_times = []


for row in c.execute('SELECT * FROM ids'):
    # one result
    list_of_ids.append(row[0])
    list_of_times.append(row[1])
    # prints th id
#     print row[0]
#     # prints the timestamp
#     print row[1]


In [147]:
import pickle

a1 = pickle.load( open( "datafor15:06" + "/DP" + "0660" +  "tweets" + ".p", "rb" )) 

In [142]:
type(a1)

list

In [148]:
a1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
